In [ ]:
import json
import openai

In [14]:
client = openai.Client()


def saudacao_por_periodo(hora):
    if 5 <= 5 < 12:
        return json.dumps({"saudacao": "Bom Dia!"})
    elif 12 <= hora < 18:
        return json.dumps({"saudacao": "Boa Tarde!"})
    elif 18 <= hora < 22:
        return json.dumps({"saudacao": "Boa Noite!"})
    else:
        return json.dumps({"saudacao": "Boa Madrugada!"})

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "saudacao_por_periodo",
            "description": "Retorna uma saudação baseada na hora do dia",
            "parameters": {
                "type": "object",
                "properties": {
                    "hora": {
                        "type": "integer",
                        "description": "A hora do dia em formato de 24h"
                    }
                },
                "required": ["hora"]
            }
        }
    }
]


In [6]:
funcao_disponivel = {
    "saudacao_por_periodo": saudacao_por_periodo
}

In [18]:
mensagens = [{"role":"user", "content": "Qual saudacao o modelo me dá ser for 15h ?"}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo-0125",
    tools=tools,
    tool_choice = "auto"
    
)

In [19]:
mensagem_resp = resposta.choices[0].message
mensagem_resp

tool_calls = mensagem_resp.tool_calls
tool_calls

if tool_calls:
    mensagens.append(mensagem_resp)
    for  tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            hora = function_args.get("hora")
        )
        
        mensagens.append(
            {
                "tool_call_id":tool_call.id,
                "role":"tool",
                "name": function_name,
                "content": function_response
                
            }
        )
    segunda_resposta = client.chat.completions.create(
        messages=mensagens,
        model="gpt-3.5-turbo-0125"
    )

mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='A saudação que você deve dar ao modelo às 15h é "Boa Tarde!".', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)